In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Input
from tensorflow.keras.optimizers import Adam
from scikeras.wrappers import KerasClassifier
from sklearn.metrics import accuracy_score, classification_report

2024-06-10 21:23:43.151750: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-06-10 21:23:45.152958: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-06-10 21:23:49.152695: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-10 21:24:07.444253: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
# Load data
data = pd.read_csv('train.csv')

In [3]:
# Ensure all feature columns are numeric
X = data.drop(columns=['nforest_type'])
X = X.apply(pd.to_numeric, errors='coerce')
y = data['nforest_type']

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Fit the label encoder on the combined set of labels
label_encoder = LabelEncoder()
y_combined = pd.concat([y_train, y_test], axis=0)
label_encoder.fit(y_combined)

# Transform the labels
y_train_encoded = label_encoder.transform(y_train)
y_test_encoded = label_encoder.transform(y_test)

# Normalize data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

num_classes = len(label_encoder.classes_)

In [4]:
def create_best_model():
    model = Sequential()
    model.add(Input(shape=(X_train.shape[1],)))
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(num_classes, activation='softmax'))
    model.compile(optimizer=Adam(learning_rate=0.001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

In [ ]:
# def create_best_model(input_shape, num_classes):
#     model = Sequential()
#     model.add(Input(shape=(input_shape,)))  # Input Layer
    
#     model.add(Dense(512, activation='relu'))  # First Dense Layer
#     model.add(Dropout(0.2))  # First Dropout Layer
    
#     model.add(Dense(512, activation='relu'))  # Second Dense Layer
#     model.add(Dropout(0.2))  # Second Dropout Layer
    
#     model.add(Dense(512, activation='relu'))  # Third Dense Layer
#     model.add(Dropout(0.2))  # Third Dropout Layer
    
#     model.add(Dense(512, activation='relu'))  # Fourth Dense Layer
#     model.add(Dropout(0.2))  # Fourth Dropout Layer
    
#     model.add(Dense(512, activation='relu'))  # Fifth Dense Layer
#     model.add(Dropout(0.2))  # Fifth Dropout Layer
    
#     model.add(Dense(512, activation='relu'))  # Sixth Dense Layer
#     model.add(Dropout(0.2))  # Sixth Dropout Layer
    
#     model.add(Dense(512, activation='relu'))  # Seventh Dense Layer
#     model.add(Dropout(0.2))  # Seventh Dropout Layer
    
#     model.add(Dense(num_classes, activation='softmax'))  # Output Layer
    
#     model.compile(optimizer=Adam(learning_rate=0.001), 
#                   loss='sparse_categorical_crossentropy', 
#                   metrics=['accuracy'])
    
#     return model

In [5]:
# Create the model with the best parameters
best_model = create_best_model()

# Train the model with the best parameters
best_model.fit(X_train, y_train_encoded, epochs=10, batch_size=16, validation_data=(X_test, y_test_encoded))

2024-06-10 21:24:27.744927: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2251] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


Epoch 1/10
653/653 ━━━━━━━━━━━━━━━━━━━━ 3s 916us/step - accuracy: 0.5269 - loss: 0.9669 - val_accuracy: 0.6296 - val_loss: 0.8372
Epoch 2/10
653/653 ━━━━━━━━━━━━━━━━━━━━ 0s 714us/step - accuracy: 0.5936 - loss: 0.8686 - val_accuracy: 0.6530 - val_loss: 0.8063
Epoch 3/10
653/653 ━━━━━━━━━━━━━━━━━━━━ 1s 719us/step - accuracy: 0.6063 - loss: 0.8409 - val_accuracy: 0.6660 - val_loss: 0.7667
Epoch 4/10
653/653 ━━━━━━━━━━━━━━━━━━━━ 0s 716us/step - accuracy: 0.6284 - loss: 0.8269 - val_accuracy: 0.6664 - val_loss: 0.7569
Epoch 5/10
653/653 ━━━━━━━━━━━━━━━━━━━━ 0s 707us/step - accuracy: 0.6417 - loss: 0.8007 - val_accuracy: 0.6702 - val_loss: 0.7451
Epoch 6/10
653/653 ━━━━━━━━━━━━━━━━━━━━ 0s 680us/step - accuracy: 0.6395 - loss: 0.8093 - val_accuracy: 0.6752 - val_loss: 0.7374
Epoch 7/10
653/653 ━━━━━━━━━━━━━━━━━━━━ 0s 685us/step - accuracy: 0.6408 - loss: 0.7900 - val_accuracy: 0.6821 - val_loss: 0.7357
Epoch 8/10
653/653 ━━━━━━━━━━━━━━━━━━━━ 0s 676us/step - accuracy: 0.6463 - loss: 0.7790 - 

In [5]:
# # Wrap the model using the KerasClassifier
# model = KerasClassifier(model=create_model, epochs=50, batch_size=32, verbose=0, input_dim=13)

# # Define the grid of hyperparameters to search
# param_grid = {
#     'model__neurons1': [32, 64, 128],
#     'model__neurons2': [16, 32, 64],
#     'model__learning_rate': [0.01, 0.001, 0.0001],
#     'batch_size': [16, 32, 64]
# }

# # Create the GridSearchCV object with verbose set to 2
# grid = GridSearchCV(estimator=model, param_grid=param_grid, cv=5, verbose=2)

# # Fit the GridSearchCV object to the data
# grid_result = grid.fit(X_train, y_train)

# # Summarize the results
# print(f"Best: {grid_result.best_score_} using {grid_result.best_params_}")

Fitting 5 folds for each of 81 candidates, totalling 405 fits


2024-06-06 15:02:54.768245: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2251] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


[CV] END batch_size=16, model__learning_rate=0.01, model__neurons1=32, model__neurons2=16; total time=  20.0s
[CV] END batch_size=16, model__learning_rate=0.01, model__neurons1=32, model__neurons2=16; total time=  15.8s
[CV] END batch_size=16, model__learning_rate=0.01, model__neurons1=32, model__neurons2=16; total time=  16.0s
[CV] END batch_size=16, model__learning_rate=0.01, model__neurons1=32, model__neurons2=16; total time=  15.8s
[CV] END batch_size=16, model__learning_rate=0.01, model__neurons1=32, model__neurons2=16; total time=  15.9s
[CV] END batch_size=16, model__learning_rate=0.01, model__neurons1=32, model__neurons2=32; total time=  15.8s
[CV] END batch_size=16, model__learning_rate=0.01, model__neurons1=32, model__neurons2=32; total time=  15.7s
[CV] END batch_size=16, model__learning_rate=0.01, model__neurons1=32, model__neurons2=32; total time=  16.2s
[CV] END batch_size=16, model__learning_rate=0.01, model__neurons1=32, model__neurons2=32; total time=  16.0s
[CV] END b

In [26]:
#  {'batch_size': 16, 
#   'model__learning_rate': 0.001, 
#   'model__neurons1': 128,
#   'model__neurons2': 64}

# # Train the best model
# best_model = grid_result.best_estimator_
# best_model.fit(X_train, y_train)

# # Predict on the test set
# y_pred = best_model.predict(X_test)

In [ ]:
"""
200 epochs : 0.68718
300 epochs : 
400 epochs : 
"""

In [6]:
# Load new test data
test_data = pd.read_csv('test.csv')

# Normalize data (using the same scaler as for training data)
test = scaler.transform(test_data)

# Predict the new test set
y_new_pred_prob = best_model.predict(test)
y_new_pred = np.argmax(y_new_pred_prob, axis=1)

# Decode the predicted labels to their original form
y_new_pred_decoded = label_encoder.inverse_transform(y_new_pred)

# Create a DataFrame for the submission
submission = pd.DataFrame({
    'id': test_data.index,
    'nforest_type': y_new_pred_decoded
})

125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 455us/step


In [7]:
# Load sample submission file
sample_submission = pd.read_csv('sample_submission.csv')

# Create submission DataFrame
submission = pd.DataFrame({'id': sample_submission['id'], 'nforest_type': y_new_pred_decoded})

# # Save the submission DataFrame to a CSV file
submission.to_csv('MLPs-Feat-1000.csv', index=False)
submission

,id,nforest_type
0,13467,DEF
1,12719,DDF
2,1054,MDF
3,13747,DDF
4,9453,DDF
...,...,...
3995,115,MDF
3996,10654,MDF
3997,5718,DDF
3998,13054,MDF


In [8]:
count_MDF = submission['nforest_type'].value_counts().get('MDF', 0)
count_DDF = submission['nforest_type'].value_counts().get('DDF', 0)
count_DEF = submission['nforest_type'].value_counts().get('DEF', 0)

print(f'MDF : {count_MDF}')
print(f'DDF : {count_DDF}')
print(f'DEF : {count_DEF}')

MDF : 1977
DDF : 1304
DEF : 719


In [9]:
XGBoost = pd.read_csv('XGBoost-feature-Newhyperparameters.csv')
MDF = XGBoost['nforest_type'].value_counts().get('MDF', 0)
DDF = XGBoost['nforest_type'].value_counts().get('DDF', 0)
DEF = XGBoost['nforest_type'].value_counts().get('DEF', 0)

print(f'MDF : {MDF}')
print(f'DDF : {DDF}')
print(f'DEF : {DEF}')

MDF : 1755
DDF : 1421
DEF : 824
